In [ ]:
## 準備
!pip install boto3 botocore awscli opencv-python scikit-image python-dotenv

In [ ]:
## Googleドライブをマウント
from google.colab import drive
drive.mount('./drive')

In [ ]:
## 環境変数設定
import dotenv
dotenv.load_dotenv('./drive/MyDrive/bedrock_iam_key.env')

In [ ]:
import boto3
bedrock = boto3.client(service_name='bedrock', region_name='us-west-2')

bedrock.list_foundation_models()

In [ ]:
## キーフレーズのリスト
keyphrases = [
    'Python',
    'Machine Learning',
    # 'Study group',
    'Niigata',
]

In [ ]:
## 雰囲気に関わるリスト
atmospheres = [
    'Cool',
    'Stylish',
    'Heartwarming',
    'Warm',
]

In [ ]:
import random
phrase = f'{",".join(keyphrases)}, {",".join(random.sample(atmospheres, 2))}, White background'

prompt_data = f"""\
Please create a logo image based on the following phrase, with the words "PyML Niigata" in the image.

{phrase}
"""

In [ ]:
import boto3
import json
import base64
from IPython.display import HTML

bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)
cnt = 1

body = json.dumps({
  "text_prompts": [
    {
      "text": prompt_data
    }
  ],
  # 'style_preset': 'digital-art',
  "cfg_scale": 7,
  "seed": 0,
  "steps": 30
})
modelId = "stability.stable-diffusion-xl-v0"  ## Stable Diffusion XL v0.8
accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType,
)
response_body = json.loads(response.get("body").read())
print(response_body['result'])

In [ ]:
# 取得した画像データの表示
base64_image_data = response_body.get("artifacts")[0].get("base64")
## HTMLを生成して画像を表示
image_html = f'<img src="data:image/jpeg;base64,{base64_image_data}" />'
display(HTML(image_html))

In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import display_png

image_data = base64.b64decode(base64_image_data)
image_stream = BytesIO(image_data)
image = Image.open(image_stream)
path=f"./drive/MyDrive/pyml_logo_sd_{cnt}.png"
image.save(path)

display_png(Image.open(path))


In [ ]:
print(prompt_data)